In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash
import mysql.connector

app = Flask(__name__)
app.secret_key = 'peter930512'  # 用于会话管理的密钥

# 配置你的 MySQL 连接
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='peter930512',
    database='專題用'
)

@app.route('/', methods=['GET', 'POST'])
def manage_data():
    cursor = conn.cursor(dictionary=True)

    # 用户相关操作
    if request.method == 'POST':
        if 'add' in request.form:  # 检查是否为添加用户请求
            name = request.form['name']
            email = request.form['email']
            cursor.execute("INSERT INTO Users (Name, Email) VALUES (%s, %s)", (name, email))
            conn.commit()
            flash('用户添加成功!')
        elif 'update' in request.form:  # 更新用户请求
            user_id = request.form['user_id']
            name = request.form['name']
            email = request.form['email']
            cursor.execute("UPDATE Users SET Name = %s, Email = %s WHERE UserID = %s", (name, email, user_id))
            conn.commit()
            flash('用户信息更新成功!')
        elif 'delete' in request.form:  # 删除用户请求
            user_id = request.form['user_id']
            cursor.execute("DELETE FROM Users WHERE UserID = %s", (user_id,))
            conn.commit()
            flash('用户删除成功!')

        # 推荐项相关操作
        elif 'add_rec' in request.form:  # 添加推荐项
            user_id = request.form['rec_user_id']
            product_id = request.form['product_id']
            recommendation_reason = request.form['recommendation_reason']
            cursor.execute("INSERT INTO Recommendations (UserID, ProductID, RecommendationReason) VALUES (%s, %s, %s)",
                           (user_id, product_id, recommendation_reason))
            conn.commit()
            flash('推荐项添加成功!')
        elif 'update_rec' in request.form:  # 更新推荐项
            rec_id = request.form['rec_id']
            user_id = request.form['rec_user_id']
            product_id = request.form['product_id']
            recommendation_reason = request.form['recommendation_reason']
            cursor.execute("UPDATE Recommendations SET UserID = %s, ProductID = %s, RecommendationReason = %s WHERE RecommendationID = %s",
                           (user_id, product_id, recommendation_reason, rec_id))
            conn.commit()
            flash('推荐项更新成功!')
        elif 'delete_rec' in request.form:  # 删除推荐项
            rec_id = request.form['rec_id']
            cursor.execute("DELETE FROM Recommendations WHERE RecommendationID = %s", (rec_id,))
            conn.commit()
            flash('推荐项删除成功!')

        # 反馈项相关操作
        elif 'add_feedback' in request.form:  # 添加反馈项
            user_id = request.form['FeedbackID']
            feedback_text = request.form['Comments']
            cursor.execute("INSERT INTO Feedback (UserID, Comments) VALUES (%s, %s)", (user_id, feedback_text))
            conn.commit()
            flash('反馈项添加成功!')
        elif 'update_feedback' in request.form:  # 更新反馈项
            feedback_id = request.form['FeedbackID']
            user_id = request.form['UserID']
            feedback_text = request.form['Comments']
            cursor.execute("UPDATE Feedback SET UserID = %s, Comments = %s WHERE FeedbackID = %s",
                           (user_id, feedback_text, feedback_id))
            conn.commit()
            flash('反馈项更新成功!')
        elif 'delete_feedback' in request.form:  # 删除反馈项
            feedback_id = request.form['FeedbackID']
            cursor.execute("DELETE FROM Feedback WHERE FeedbackID = %s", (feedback_id,))
            conn.commit()
            flash('反馈项删除成功!')

    # 查询所有用户
    cursor.execute("SELECT Users.UserID, Users.Name, Users.Email FROM Users;")
    user_results = cursor.fetchall()

    # 查询所有推荐项
    cursor.execute("""
        SELECT Recommendations.RecommendationID, Recommendations.UserID, Recommendations.ProductID,
               Recommendations.RecommendationReason
        FROM Recommendations
        INNER JOIN Users ON Recommendations.UserID = Users.UserID
        INNER JOIN Insurance_Products ON Recommendations.ProductID = Insurance_Products.ProductID
    """)
    rec_results = cursor.fetchall()

    # 查询所有反馈项
    cursor.execute("""
        SELECT Feedback.FeedbackID, Feedback.UserID, Feedback.Rating, Feedback.Comments
        FROM Feedback
        INNER JOIN Users ON Feedback.UserID = Users.UserID
    """)
    feedback_results = cursor.fetchall()

    # INNER JOIN 查询（根据用户信息和推荐项进行筛选）
    name_filter = request.args.get('name_filter', '')  # 从GET请求接收姓名筛选条件
    product_filter = request.args.get('product_filter', '')  # 从GET请求接收产品名称筛选条件
    user_id_filter = request.args.get('user_id_filter', '')  # 从GET请求接收 UserID 筛选条件
    email_filter = request.args.get('email_filter', '')  # 从GET请求接收 Email 筛选条件
    query_params = []

    sql_query = """
        SELECT Users.UserID, Users.Name, Users.Email, Insurance_Products.ProductName, Recommendations.RecommendationID,
               Recommendations.RecommendationReason, Chatbot_Interactions.UserQuery, Chatbot_Interactions.BotResponse
        FROM Users
        INNER JOIN Recommendations ON Users.UserID = Recommendations.UserID
        INNER JOIN Insurance_Products ON Recommendations.ProductID = Insurance_Products.ProductID
        INNER JOIN Chatbot_Interactions ON Users.UserID = Chatbot_Interactions.UserID
    """
    filters = []

    if name_filter:  # 如果有提供姓名筛选
        filters.append("Users.Name LIKE %s")
        query_params.append(f"%{name_filter}%")  # 使用模糊匹配

    if product_filter:  # 如果有提供产品名称筛选
        filters.append("Insurance_Products.ProductName LIKE %s")
        query_params.append(f"%{product_filter}%")  # 使用模糊匹配

    if user_id_filter:  # 如果有提供 UserID 筛选
        filters.append("Users.UserID = %s")
        query_params.append(user_id_filter)

    if email_filter:  # 如果有提供 Email 筛选
        filters.append("Users.Email LIKE %s")
        query_params.append(f"%{email_filter}%")  # 使用模糊匹配

    if filters:  # 如果有任何过滤条件，则加上 WHERE 子句
        sql_query += " WHERE " + " AND ".join(filters)

    cursor.execute(sql_query, query_params)
    join_results = cursor.fetchall()
    cursor.close()

    return render_template('template4.html', user_results=user_results, rec_results=rec_results, feedback_results=feedback_results, join_results=join_results)

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.63.8:5000
Press CTRL+C to quit
192.168.63.8 - - [23/Oct/2024 11:16:53] "GET / HTTP/1.1" 200 -
192.168.63.8 - - [23/Oct/2024 11:17:15] "POST / HTTP/1.1" 500 -
Traceback (most recent call last):
  File "C:\Users\peter_4bywd5d\anaconda3\Lib\site-packages\mysql\connector\connection_cext.py", line 705, in cmd_query
    self._cmysql.query(
_mysql_connector.MySQLInterfaceError: Cannot add or update a child row: a foreign key constraint fails (`專題用`.`feedback`, CONSTRAINT `feedback_ibfk_1` FOREIGN KEY (`UserID`) REFERENCES `users` (`UserID`))

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\peter_4bywd5d\anaconda3\Lib\site-packages\flask\app.py", line 2552, in __call__
    return self.wsgi_app(environ, start_response)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\peter_4bywd5d\anaconda3\Lib\site-pac